# Group Details

## Group Name:

### Student 1:

### Student 2:

### Student 3:

# Loading Data and Preliminaries

In [ ]:
%cd /content/drive/MyDrive/TUe/DeepLearning/Assignment2

/content/drive/MyDrive/TUe/DeepLearning/Assignment2


In [ ]:
import os

import matplotlib
import matplotlib.pyplot as plt

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm
from glob import glob
from sklearn.model_selection import train_test_split

In [ ]:
def load_array(filename, task):
    datapoint = np.load(filename)
    if task == 'task 1':
        initial_state = datapoint['initial_state']
        terminal_state = datapoint['terminal_state']
        return initial_state, terminal_state
    elif task == 'task 2' or task == 'task 3':
        whole_trajectory = datapoint['trajectory']
        # change shape: (num_bodies, attributes, time) ->  num_bodies, time, attributes
        whole_trajectory = np.swapaxes(whole_trajectory, 1, 2)
        initial_state = whole_trajectory[:, 0]
        target = whole_trajectory[:, 1:, 1:]  # drop the first timepoint (second dim) and mass (last dim) for the prediction task
        return initial_state, target
    else:
        raise NotImplementedError("'task' argument should be 'task 1', 'task 2' or 'task 3'!")


# Data Handling and Preprocessing

In [218]:
def pad_array(data):
    # Pad the array with zeros if necessary to have 9 rows
    padded_input_data  = np.pad(data[0], ((0, 9 - data[0].shape[0]), (0, 0)), mode='constant')
    padded_target_data = np.pad(data[1], ((0, 9 - data[1].shape[0]), (0, 0)), mode='constant')
    return torch.tensor(padded_input_data, dtype=torch.float32), torch.tensor(padded_target_data, dtype=torch.float32)

def create_mask(data,padded_input_data):
    # Create a boolean mask array indicating the padded rows
    mask = np.ones_like(padded_input_data, dtype=bool)
    mask[data[0].shape[0]:] = False
    return torch.tensor(mask, dtype=torch.bool)

def pair_values(data):
    # Pair up the values of the same columns from every row with every other row
    num_rows, num_cols = data.shape
    padded_input_data = torch.zeros((num_rows, num_rows, num_cols, 2))

    for i in range(num_rows):
        pair_idx = 0
        for j in range(num_rows):

            padded_input_data[i, pair_idx, :, 0] = data[i]
            padded_input_data[i, pair_idx, :, 1] = data[j]
            pair_idx += 1

    return padded_input_data

def get_euclidean_distance(x):
    euclidean_distances = torch.zeros((len(x), len(x)))
    for i in range(len(x)):
        source_x, source_y= x[i][1], x[i][2]
        #euclidean_distance=[]
        for j in range(len(x)):
            target_x, target_y= x[j][1], x[j][2]
            euclidean_distances[i][j]=np.sqrt((source_x-target_x)**2+(source_y-target_y)**2)
            #euclidean_distance.append(np.sqrt((source_x-target_x)**2+(source_y-target_y)**2))
        #euclidean_distances.append(euclidean_distance)

    return euclidean_distances #list of lists

def target_difference(target_x, source_x):
    # target x and source x are of shape (9, 2)
    #Subtract the x and y coordinates of the target from the source
    return target_x - source_x


def process_file(file_path):
    # Main function to process a single file
    data = load_array(file_path, 'task 1')
    padded_input_data, padded_target_data = pad_array(data)
    mask = create_mask(data,padded_input_data)
    input_data = pair_values(padded_input_data)
    euclidean_distance = get_euclidean_distance(padded_input_data)
    target_data= target_difference(padded_target_data, padded_input_data[:,1:3])

    return input_data, target_data, padded_input_data, mask, euclidean_distance


In [219]:
class CustomDataset_2(Dataset):
    def __init__(self, folder):
        self.folder = folder
        self.file_list = os.listdir(folder)

    def __len__(self):
        return len(self.file_list)

    def _read_file(self, file_path):
        input_data, target_data,data, mask, distances= process_file(file_path)
        return input_data, target_data,data, mask, distances

    def __getitem__(self, index):
        file_path = os.path.join(self.folder, self.file_list[index])
        # Read and preprocess the data from the file
        input_data, target_data,data, mask, distance = self._read_file(file_path)
        # Return the preprocessed data
        return input_data, target_data,data, mask, distance

# Model Implementation

## NPE implementation

In [220]:
class NPEEncoder(nn.Module):
    def __init__(self, hidden_units):
        super(NPEEncoder, self).__init__()
        self.pairwise_layer = nn.Linear(10, hidden_units, bias=False)
        self.feedforward = nn.Sequential(
            nn.Linear(hidden_units, 50, bias=False),
            nn.ReLU(),
            nn.Linear(50, 50, bias=False),
            nn.ReLU(),
            nn.Linear(50, 50, bias=False),
            nn.ReLU(),
            nn.Linear(50, 50, bias=False),
            nn.ReLU()
        )

    def forward(self, x):
        x= x.to(torch.float32)
        x = self.pairwise_layer(x)
        x = self.feedforward(x)
        return x

class NPEDecoder(nn.Module):
    def __init__(self):
        super(NPEDecoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.Linear(55, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 2)
        )

    def forward(self, x):
        x = self.decoder(x)
        return x

In [221]:
class NPEModel(nn.Module):
    def __init__(self, hidden_units, device):
        super(NPEModel, self).__init__()
        self.encoder = NPEEncoder(hidden_units)
        self.decoder = NPEDecoder()
        self.device = device

    def forward(self, x,unpaired_data, mask, distance, neighborhood_threshold=5):
        output=[]
        dist_mask = ((distance > 0) & (distance < neighborhood_threshold))
        for batch in range(x.size(0)):
            batch_data= x[batch]
            decoder_input = []
            for body in range(batch_data.size(0)):
                focus_body= unpaired_data[batch][body]

                if dist_mask[batch][body].any():
                    chunk = torch.flatten(x[batch][body][dist_mask[batch][body]],start_dim=1)
                    neighbor_encodings = torch.sum(self.encoder(chunk), dim=0)
                else:
                    neighbor_encodings = torch.zeros(50).to(self.device)
                # Concatenate the focus body encoding with the focus_body
                decoder_input.append(torch.cat((neighbor_encodings, focus_body), dim=0))

            # Decode the concatenated vector
            decoded = self.decoder(torch.stack(decoder_input))
            output.append(decoded)


        return output

# Model Training

## NPE training

In [223]:
def load_data(folder):
    file_list = os.listdir(folder)
    output = []

    for filename in file_list:
        file_path = os.path.join(folder, filename)
        output.append(process_file(file_path))

    return output

In [224]:
data_dir = 'data/task 1/train'
# dataset = CustomDataset_2(data_dir)

dataset = load_data(data_dir)


data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

In [225]:
test_data_dir = 'data/task 1/test'
# test_dataset = CustomDataset_2(test_data_dir)

test_dataset = load_data(test_data_dir)


test_data_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [226]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
device

device(type='cpu')

In [227]:
# Hyperparameters
hidden_units = 25
learning_rate = 1e-3
learning_rate_decay = 0.99
iterations = 50
batch_size = 100

# Create the model
model = NPEModel(hidden_units, device=device)
model.to(device)
criterion = nn.MSELoss()
# optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [228]:
def train_sim_model(model, train_data_loader, valid_data_loader, optimizer, criterion, device=torch.device('cpu'), iterations=50):
    for iteration in tqdm(range(iterations)):
        model.train()
        train_loss=0.0
        for input_data, target_data,unpaired_data, mask, distance in train_data_loader:
        # Training steps
            input_data = input_data.to(device)
            unpaired_data = unpaired_data.to(device)
            mask = mask.to(device)
            distance = distance.to(device)
            target_data=target_data.to(device)

            optimizer.zero_grad()
            output = model(input_data, unpaired_data, mask, distance)
            output = torch.stack(output)

            loss = criterion(output, target_data)
            train_loss += loss.item()
            loss.backward()
            optimizer.step()

        train_loss /= len(train_data_loader)


        model.eval()
        test_loss=0.0
        with torch.no_grad():
            for input_data, target_data,unpaired_data, mask, distance in valid_data_loader:
                input_data = input_data.to(device)
                target_data = target_data.to(device)
                unpaired_data = unpaired_data.to(device)
                mask = mask.to(device)
                distance = distance.to(device)

                output = model(input_data, unpaired_data, mask, distance)
                output = torch.stack(output)
                test_loss += criterion(output, target_data).item()
        test_loss /= len(valid_data_loader)


            # Print loss for monitoring
            #if (iteration + 1) % 10000 == 0:
        print(f"Epoch [{iteration+1}/{iterations}], Loss: {train_loss}, Test Loss: {test_loss}")

In [229]:
train_sim_model(model, data_loader, test_data_loader, optimizer, criterion, device=device, iterations=iterations)

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch [1/50], Loss: 9.939079666137696, Test Loss: 7.648855209350586
Epoch [2/50], Loss: 8.725012143452963, Test Loss: 5.483811616897583
Epoch [3/50], Loss: 6.850018787384033, Test Loss: 4.3958611488342285
Epoch [4/50], Loss: 6.19133555094401, Test Loss: 4.082082748413086
Epoch [5/50], Loss: 5.726301956176758, Test Loss: 3.9446059465408325
Epoch [6/50], Loss: 6.038074620564779, Test Loss: 3.8227548599243164
Epoch [7/50], Loss: 5.315569623311361, Test Loss: 3.4849575757980347
Epoch [8/50], Loss: 5.040620136260986, Test Loss: 3.281022310256958
Epoch [9/50], Loss: 4.7967591444651285, Test Loss: 3.218953013420105
Epoch [10/50], Loss: 4.6282915592193605, Test Loss: 2.973642945289612
Epoch [11/50], Loss: 4.3582629839579266, Test Loss: 2.850334882736206
Epoch [12/50], Loss: 4.26755428314209, Test Loss: 2.8038569688796997
Epoch [13/50], Loss: 4.1107229868570965, Test Loss: 2.7229620218276978
Epoch [14/50], Loss: 3.9689907868703207, Test Loss: 2.653095841407776
Epoch [15/50], Loss: 3.89614974657

# Task 2

In [231]:
def files_2_data(files):
    output = []
    for file_path in tqdm(files):
        datapoint = np.load(file_path)
        whole_trajectory = datapoint['trajectory']
        # change shape: (num_bodies, attributes, time) ->  num_bodies, time, attributes
        whole_trajectory = np.swapaxes(whole_trajectory, 1, 2)
        for i in range(whole_trajectory.shape[1] - 1):
            data = whole_trajectory[:, i], whole_trajectory[:, i + 1, 1:3]
            padded_input_data, padded_target_data = pad_array(data)
            mask = create_mask(data,padded_input_data)
            input_data = pair_values(padded_input_data)
            euclidean_distance = get_euclidean_distance(padded_input_data)
            target_data= target_difference(padded_target_data, padded_input_data[:,1:3])

            output.append((input_data, target_data, padded_input_data, mask, euclidean_distance))
    return output

In [232]:
files_rate = 0.2

train_files = glob('data/task 2_3/train/*')
train_files = train_files[:int(len(train_files) * files_rate)]

train_files, valid_files = train_test_split(train_files, train_size=0.7)

In [233]:
train_data = files_2_data(train_files)

valid_data = files_2_data(valid_files)

  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

In [234]:
len(train_data)

6125

In [235]:
BATCH_SIZE = 128

train_data_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_data_loader = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False)

In [236]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [237]:
# Hyperparameters
hidden_units = 25
LR = 1e-3
EPOCHS = 10

# Create the model
model = NPEModel(hidden_units, device=device).to(device)

criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=LR)

In [238]:
train_sim_model(model, train_data_loader, valid_data_loader, optimizer, criterion, device, iterations=EPOCHS)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch [1/10], Loss: 0.005665687662258279, Test Loss: 0.0018616489912594923
Epoch [2/10], Loss: 0.0011458572601744284, Test Loss: 0.0015075538712236184
Epoch [3/10], Loss: 0.0010063542734618143, Test Loss: 0.001485489628976211
Epoch [4/10], Loss: 0.0009744990717687566, Test Loss: 0.001460649390620264
Epoch [5/10], Loss: 0.0009574162128653066, Test Loss: 0.0014283904271327299
Epoch [6/10], Loss: 0.0009554821517667733, Test Loss: 0.001445167598748495
Epoch [7/10], Loss: 0.0009393441020317065, Test Loss: 0.0014472210869743403
Epoch [8/10], Loss: 0.0009338501028347915, Test Loss: 0.0014305383811006323
Epoch [9/10], Loss: 0.000924186944757821, Test Loss: 0.0014180150398963385
Epoch [10/10], Loss: 0.0009200674576277379, Test Loss: 0.0014225230922668495


In [239]:
def recalc_data(padded_input_data):
    input_data = []
    euclidean_distance = []
    for elem in padded_input_data:
        input_data.append(pair_values(elem))
        euclidean_distance.append(get_euclidean_distance(elem))

    return torch.stack(input_data), padded_input_data, torch.stack(euclidean_distance)

In [240]:
def simulate(init_data, times=5):
    input_data, unpaired_data, mask, distance = init_data

    input_data = input_data.to(device)
    unpaired_data = unpaired_data.to(device)
    mask = mask.to(device)
    distance = distance.to(device)

    locations = []
    model.eval()
    with torch.no_grad():
        for time in tqdm(range(times)):

            output = model(input_data, unpaired_data, mask, distance)

            unpaired_data[:,:,3:5] = torch.stack([torch.stack(ten) for ten in output])
            unpaired_data[:,:,1:3] += unpaired_data[:,:,3:5]
            locations.append(unpaired_data[:,:,1:3])

            input_data, unpaired_data, distance = recalc_data(unpaired_data)


    return locations

# Evaluation

In [ ]:
#todo